In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 21 06:04:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!pip3 install tokenizers wandb sentencepiece
!pip3 install transformers huggingface-hub
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 30.3 MB/s 
     |████████████████████████████████| 1.9 MB 89.5 MB/s 
     |████████████████████████████████| 1.3 MB 85.8 MB/s 
     |████████████████████████████████| 174 kB 98.5 MB/s 
     |████████████████████████████████| 182 kB 96.6 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 173 kB 64.0 MB/s 
     |████████████████████████████████| 168 kB 98.9 MB/s 
     |████████████████████████████████| 168 kB 104.5 MB/s 
     |████████████████████████████████| 166 kB 110.5 MB/s 
     |████████████████████████████████| 166 kB 91.8 MB/s 
     |████████████████████████████████| 162 kB 108.4 MB/s 
     |████████████████████████████████| 162 kB 107.1 MB/s 
     |████████████████████████████████| 158 kB 87.5 MB/s 
     |████████████████████████████████| 157 kB 116.6 MB/s 
     |██████████████████████

In [6]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('dacon')
os.chdir('sentence_classification')

In [7]:
OUTPUT_DIR = './inference/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [8]:
class CFG:
    wandb=True
    competition='sentence-classification'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="klue/roberta-large"
    path="./outputs/"
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50
    batch_size=8
    fc_dropout=0.1
    target_size=1 
    max_len=512
    max_grad_norm=1000
    seed=42
    n_fold=10
    trn_fold=[0, 1, 2, 3, 4, 5, 6]
    train=False
    use_rnn=False
    bidirectional=True


if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [9]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Upstage-Inference', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="inference",
                     anonymous=anony)

login to wandb


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [10]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
from joblib import Parallel, delayed
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm

# Sklearn Imports
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_curve, auc

# PyTorch
import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter, LSTM
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import torch.cuda.amp as amp


import pickle as pickle


# For Transformer Models
import transformers
from transformers import DataCollatorWithPadding

import tokenizers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.13.0+cu116
tokenizers.__version__: 0.13.2
transformers.__version__: 4.25.1
env: TOKENIZERS_PARALLELISM=true


# Utils

In [11]:
# ====================================================
# Utils
# ====================================================

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [12]:
def compute_metrics(preds_type, preds_polarity, preds_tense, preds_certainty, labels_type, labels_polarity, labels_tense, labels_certainty):
    """ validation을 위한 metrics function """
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # probs = pred.predictions

    # calculate accuracy using sklearn's function
    f1_score1 = f1_score( labels_type, preds_type, average='weighted' )
    f1_score2 = f1_score( labels_polarity, preds_polarity, average='weighted' )
    f1_score3 = f1_score( labels_tense, preds_tense, average='weighted' )
    f1_score4 = f1_score( labels_certainty, preds_certainty, average='weighted' )

    score = (f1_score1 + f1_score2 + f1_score3 + f1_score4) / 4

    return score

# Tokenizer

In [13]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.path + 'tokenizer', use_fast=True)
CFG.tokenizer = tokenizer

# Data Loading

In [14]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
submission = pd.read_csv('./sample_submission.csv')

In [15]:
LABEL_MAPPING1 = {"사실형": 0, "추론형": 1, "대화형": 2, "예측형": 3} # 사실형, 추론형, 대화형, 예측형
LABEL_MAPPING2 = {"긍정": 0, "부정": 1, "미정": 2} # 긍정, 부정, 미정
LABEL_MAPPING3 = {"과거": 0, "현재": 1, "미래": 2} # 과거, 현재, 미래
LABEL_MAPPING4 = {"확실": 0, "불확실": 1} # 확실, 불확실

In [16]:
train['label1']  = train['유형'].map(lambda x : LABEL_MAPPING1[x])
train['label2']  = train['극성'].map(lambda x : LABEL_MAPPING2[x])
train['label3']  = train['시제'].map(lambda x : LABEL_MAPPING3[x])
train['label4']  = train['확실성'].map(lambda x : LABEL_MAPPING4[x])

In [17]:
display(train.head())
print(train.info())
print(train.shape)

,ID,문장,유형,극성,시제,확실성,label,label1,label2,label3,label4
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0,0,2,0
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16541 entries, 0 to 16540
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      16541 non-null  object
 1   문장      16541 non-null  object
 2   유형      16541 non-null  object
 3   극성      16541 non-null  object
 4   시제      16541 non-null  object
 5   확실성     16541 non-null  object
 6   label   16541 non-null  object
 7   label1  16541 non-null  int64 
 8   label2  16541 non-null  int64 
 9   label3  16541 non-null  int64 
 10  label4  16541 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 1.4+ MB
None
(16541, 11)


In [18]:
# init columns with dummy values

test['label1']  = 0
test['label2']  = 0
test['label3']  = 0
test['label4']  = 0

In [19]:
display(test.head())
print(test.info())
print(test.shape)

,ID,문장,label1,label2,label3,label4
0,TEST_0000,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정...",0,0,0,0
1,TEST_0001,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다.",0,0,0,0
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.,0,0,0,0
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...,0,0,0,0
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7090 entries, 0 to 7089
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7090 non-null   object
 1   문장      7090 non-null   object
 2   label1  7090 non-null   int64 
 3   label2  7090 non-null   int64 
 4   label3  7090 non-null   int64 
 5   label4  7090 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 332.5+ KB
None
(7090, 6)


# Max length

In [20]:
# max length
lengths_dict = {}

for text_col in ['ID', '문장']:
    lengths = []
    tk0 = tqdm(test[text_col].fillna("").values, total=len(test))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths

CFG.max_len = max(lengths_dict['ID']) + max(lengths_dict['문장']) + 2 # CLS + SEP

LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/7090 [00:00<?, ?it/s]

  0%|          | 0/7090 [00:00<?, ?it/s]

max_len: 190
INFO:__main__:max_len: 190


# Dataset

In [21]:
def prepare_input(cfg, text):
    # text = text[:cfg.essay_max_len]
    inputs = cfg.tokenizer(text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            return_attention_mask=True,
            return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class Sentence_Dataset(torch.utils.data.Dataset):
    """ Dataset 구성을 위한 class."""
    def __init__(self, cfg, texts, labels1, labels2, labels3, labels4):
      self.cfg = cfg
      self.texts = texts
      self.labels1 = labels1
      self.labels2 = labels2
      self.labels3 = labels3
      self.labels4 = labels4

    def __getitem__(self, idx):
      input_ids = prepare_input(self.cfg, self.texts[idx])
      
      labels1 = torch.tensor(self.labels1[idx])
      labels2 = torch.tensor(self.labels2[idx])
      labels3 = torch.tensor(self.labels3[idx])
      labels4 = torch.tensor(self.labels4[idx])

      return input_ids, labels1, labels2, labels3, labels4

    def __len__(self):
      return len(self.labels1)

# Model

In [22]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
    # def forward(self, last_hidden_state):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        # sum_embeddings = torch.sum(last_hidden_state, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [23]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False, use_rnn=False, bidirectional=True, num_of_classes=72):
        super().__init__()
        self.cfg = cfg
        self.use_rnn = use_rnn

        self.my_hidden_dim = 256

        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)

        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        dim = self.config.hidden_size
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(dim, self.my_hidden_dim)
        self._init_weights(self.fc)

        self.head_dropout = nn.Dropout(cfg.fc_dropout)
        self.type_clf = nn.Linear(self.my_hidden_dim,4)
        self.polarity_clf = nn.Linear(self.my_hidden_dim,3)
        self.tense_clf = nn.Linear(self.my_hidden_dim,3)
        self.certainty_clf = nn.Linear(self.my_hidden_dim,2)

        # check if rnn is used
        if self.use_rnn:
            self.bidirectional = bidirectional
            self.pooling = MeanPooling()
            self.seq_encoder = LSTM(dim, int(dim / 2), 1, dropout=0.1, bidirectional=self.bidirectional, batch_first=True)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[:, 0, :]
        outputs = torch.squeeze(outputs)
        # last_hidden_states = outputs[0]
        return outputs
    
    def encode(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state
        outputs, _ = self.seq_encoder(outputs)
        return outputs


    def forward(self, inputs):
        if self.use_rnn:
            features = self.encode(inputs)
            features = self.pooling(features, inputs['attention_mask'])
        else:
            features = self.feature(inputs)
        output = self.fc(self.fc_dropout(features))
        output = self.head_dropout(output)

        output_type = self.type_clf(output)
        output_polarity = self.polarity_clf(output)
        output_tense = self.tense_clf(output)
        output_certainty = self.certainty_clf(output)

        return (output_type, output_polarity, output_tense, output_certainty)

In [24]:
from torch.nn.modules.loss import _WeightedLoss


class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    def k_one_hot(self, targets:torch.Tensor, n_classes:int, smoothing=0.0):
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                                  .fill_(smoothing /(n_classes-1)) \
                                  .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def reduce_loss(self, loss):
        return loss.mean() if self.reduction == 'mean' else loss.sum() if self.reduction == 'sum' else loss

    def forward(self, inputs, targets):
        assert 0 <= self.smoothing < 1

        targets = self.k_one_hot(targets, inputs.size(-1), self.smoothing)
        log_preds = F.log_softmax(inputs, -1)

        if self.weight is not None:
            log_preds = log_preds * self.weight.unsqueeze(0)

        return self.reduce_loss(-(targets * log_preds).sum(dim=-1))

In [25]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if name in self.backup:
                    param.data = self.backup[name]
            self.backup = {}


# Inference

In [26]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def append_pred_to_list(preds, y_preds):
    pred_np = y_preds.to('cpu').numpy()
    if len(pred_np.shape) == 1:
        pred_np = np.array([pred_np.tolist()])
    preds.append(pred_np)


def inference_fn(test_loader, model, device):
    model.eval()
    
    preds_type, preds_polarity, preds_tense, preds_certainty = [], [], [], []
    
    start = end = time.time()
    for step, (inputs, labels1, labels2, labels3, labels4) in enumerate(test_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        
        with torch.no_grad():
            y_preds = model(inputs)
        y_pred_type, y_pred_polarity, y_pred_tense, y_pred_certainty = y_preds[0], y_preds[1], y_preds[2], y_preds[3]
        
        append_pred_to_list(preds_type, y_pred_type)
        append_pred_to_list(preds_polarity, y_pred_polarity)
        append_pred_to_list(preds_tense, y_pred_tense)
        append_pred_to_list(preds_certainty, y_pred_certainty)
        
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(test_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  .format(step, len(test_loader),
                          remain=timeSince(start, float(step+1)/len(test_loader))))
    predictions_type = np.concatenate(preds_type)
    predictions_polarity = np.concatenate(preds_polarity)
    predictions_tense = np.concatenate(preds_tense)
    predictions_certainty = np.concatenate(preds_certainty)

    return predictions_type, predictions_polarity, predictions_tense, predictions_certainty

In [27]:
texts = test['문장'].values
labels1 = test['label1'].values
labels2 = test['label2'].values
labels3 = test['label3'].values
labels4 = test['label4'].values

In [ ]:
test_dataset = Sentence_Dataset(CFG, texts, labels1, labels2, labels3, labels4)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

predictions_type, predictions_polarity, predictions_tense, predictions_certainty = [], [], [], []

print('config_path: ', CFG.path+"config.pth")

for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.path+"config.pth", pretrained=False, use_rnn=CFG.use_rnn, bidirectional=CFG.bidirectional)
    state = torch.load(CFG.path + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    
    model.load_state_dict(state['model'])
    model = model.to(device)
    pred_type, pred_polarity, pred_tense, pred_certainty = inference_fn(test_loader, model, device)

    predictions_type.append(pred_type)
    predictions_polarity.append(pred_polarity)
    predictions_tense.append(pred_tense)
    predictions_certainty.append(pred_certainty)

    del model, state, pred_type, pred_polarity, pred_tense, pred_certainty
    gc.collect()
    torch.cuda.empty_cache()

predictions_type = np.mean(predictions_type, axis=0)
predictions_polarity = np.mean(predictions_polarity, axis=0)
predictions_tense = np.mean(predictions_tense, axis=0)
predictions_certainty = np.mean(predictions_certainty, axis=0)

config_path:  ./outputs/config.pth
EVAL: [0/887] Elapsed 0m 3s (remain 55m 0s) 
EVAL: [100/887] Elapsed 0m 29s (remain 3m 50s) 
EVAL: [200/887] Elapsed 0m 56s (remain 3m 13s) 
EVAL: [300/887] Elapsed 1m 23s (remain 2m 41s) 
EVAL: [400/887] Elapsed 1m 49s (remain 2m 12s) 
EVAL: [500/887] Elapsed 2m 15s (remain 1m 44s) 
EVAL: [600/887] Elapsed 2m 42s (remain 1m 17s) 
EVAL: [700/887] Elapsed 3m 9s (remain 0m 50s) 
EVAL: [800/887] Elapsed 3m 35s (remain 0m 23s) 
EVAL: [886/887] Elapsed 3m 58s (remain 0m 0s) 
EVAL: [0/887] Elapsed 0m 0s (remain 8m 33s) 
EVAL: [100/887] Elapsed 0m 27s (remain 3m 32s) 
EVAL: [200/887] Elapsed 0m 54s (remain 3m 5s) 
EVAL: [300/887] Elapsed 1m 20s (remain 2m 36s) 
EVAL: [400/887] Elapsed 1m 47s (remain 2m 9s) 
EVAL: [500/887] Elapsed 2m 13s (remain 1m 43s) 
EVAL: [600/887] Elapsed 2m 40s (remain 1m 16s) 
EVAL: [700/887] Elapsed 3m 6s (remain 0m 49s) 
EVAL: [800/887] Elapsed 3m 33s (remain 0m 22s) 
EVAL: [886/887] Elapsed 3m 56s (remain 0m 0s) 
EVAL: [0/887] Ela

In [ ]:
def generate_output_preds(predictions):
    output_pred = []

    for p in predictions:
        logits = torch.Tensor(p)
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result.tolist())
    return output_pred


predictions_type = generate_output_preds(predictions_type)
predictions_polarity = generate_output_preds(predictions_polarity)
predictions_tense = generate_output_preds(predictions_tense)
predictions_certainty = generate_output_preds(predictions_certainty)

In [ ]:
test_id = test['ID']

output = pd.DataFrame({'ID':test_id,'label1': predictions_type, 'label2': predictions_polarity, 'label3': predictions_tense, 'label4': predictions_certainty})

LABEL_MAPPING1 = {0: "사실형", 1: "추론형", 2:"대화형", 3:"예측형"} # 사실형, 추론형, 대화형, 예측형
LABEL_MAPPING2 = {0: "긍정", 1: "부정", 2: "미정"} # 긍정, 부정, 미정
LABEL_MAPPING3 = {0: "과거", 1: "현재", 2: "미래"} # 과거, 현재, 미래
LABEL_MAPPING4 = {0: "확실", 1: "불확실"} # 확실, 불확실

output['n_label1']  = output['label1'].map(lambda x : LABEL_MAPPING1[x])
output['n_label2']  = output['label2'].map(lambda x : LABEL_MAPPING2[x])
output['n_label3']  = output['label3'].map(lambda x : LABEL_MAPPING3[x])
output['n_label4']  = output['label4'].map(lambda x : LABEL_MAPPING4[x])

output.head()

In [ ]:
# Reference: <https://stackoverflow.com/questions/39291499/how-to-concatenate-multiple-column-values-into-a-single-column-in-pandas-datafra>
cols = ['n_label1', 'n_label2', 'n_label3', 'n_label4']
output['label'] = output[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
output.head()

In [ ]:
output = output[['ID', 'label']]

display(output.head())
output.to_csv('./submission.csv', index=False) # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.